# IMPORTAR LIBRERÍAS Y CONFIGURAR PATH Y CLIENTE

In [1]:
# Import de las librerías, api keys y funciones
import sys
import os
from pathlib import Path
import pandas as pd
import json
from apify_client import ApifyClient
from config import APIFY_API_KEY, DB_PATH
from src.utils.database import get_db_connection, insert_dataframe_to_table

# Configuración de paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Configuración de Apify
client = ApifyClient(APIFY_API_KEY)

### Establecer las URLs del scraping de restaurantes

In [2]:
# URLs de búsqueda que scrapeará el actor de Apify
SEARCH_URLS_PARAM = [
    #"https://www.tripadvisor.es/Restaurants-g187499-oa120-Girona_Province_of_Girona_Catalonia.html", (ya scrapeada)
    #"https://www.tripadvisor.es/Restaurants-g187499-oa150-Girona_Province_of_Girona_Catalonia.html", (ya scrapeada)
    #"https://www.tripadvisor.es/Restaurants-g187499-oa180-Girona_Province_of_Girona_Catalonia.html", (ya scrapeada)
    #"https://www.tripadvisor.es/Restaurants-g187499-oa240-Girona_Province_of_Girona_Catalonia.html",(ya scrapeada)
    #"https://www.tripadvisor.es/Restaurants-g187499-oa270-Girona_Province_of_Girona_Catalonia.html" ,(ya scrapeada)
    #"https://www.tripadvisor.es/Restaurants-g187499-oa300-Girona_Province_of_Girona_Catalonia.html", (ya scrapeada)
    "https://www.tripadvisor.es/Restaurants-g187499-oa330-Girona_Province_of_Girona_Catalonia.html"
]

# DEFINIR LAS FUNCIONES PARA SCRAPEAR LOS RESTAURANTES

In [ ]:
# Definir las funciones específicas para scrapear los restaurantes a través de Apify
def run_apify_actor_slow():
    """Ejecuta el actor de Apify con configuración ultra-lenta para evitar bloqueos"""
    print("🚀 Ejecutando actor de Apify (modo lento)...")
    print(f"📋 Número de URLs: {len(SEARCH_URLS_PARAM)}")
    
    try:
        run = client.actor("maxcopell/tripadvisor").call(
            run_input={
                "startUrls": [{"url": url} for url in SEARCH_URLS_PARAM],
                "maxItems": 30,
                "proxyConfiguration": {
                    "useApifyProxy": True,
                    "apifyProxyGroups": ["RESIDENTIAL"],
                    "apifyProxyCountry": "ES"
                },
                "maxConcurrency": 1,
                "minTimeBetweenRequests": 10000,
                "searchLanguage": "es",
                "reviews": False,
            },
            wait_secs=600
        )
        
        print("📦 Recopilando datos...")
        dataset = client.dataset(run["defaultDatasetId"]).list_items().items
        
        if not dataset:
            print("❌ No se encontraron datos")
            return None
        
        print(f"✅ Se obtuvieron {len(dataset)} restaurantes")
        return dataset
        
    except Exception as e:
        print(f"❌ Error ejecutando el actor de Apify: {e}")
        return None
    
def extract_url_path(full_url):
    """Extrae solo la parte del path de la URL"""
    if full_url and 'tripadvisor.com' in full_url:
        return full_url.split('tripadvisor.com')[-1]
    return full_url

def extract_cuisines(cuisines_data):
    """Convierte cocinas a string"""
    if not cuisines_data:
        return ''
    
    if isinstance(cuisines_data, list):
        if cuisines_data and isinstance(cuisines_data[0], dict):
            return ', '.join([c.get('name', '') for c in cuisines_data])
        return ', '.join(cuisines_data)
    return str(cuisines_data)

# EJECUTAR EL SCRAPEO DE LOS RESTAURANTES Y ANALIZAR LOS DATOS

In [4]:
# Ejecutar el actor y obtener datos
dataset = run_apify_actor_slow()

🚀 Ejecutando actor de Apify (modo lento)...
📋 Número de URLs: 1
❌ Error ejecutando el actor de Apify: Monthly usage hard limit exceeded


In [5]:
# Explorar los datos
print("🔍 EXPLORACIÓN DE DATOS OBTENIDOS")
print("="*50)

# Crear DataFrame para exploración
if dataset:
    df_raw = pd.DataFrame(dataset)
    print(f"📊 Total de items: {len(df_raw)}")
    print(f"📋 Columnas: {list(df_raw.columns)}")
    
    # Mostrar primeras filas
    print("\n👀 Primeras filas:")
    display(df_raw.head())
    
    # Ver estructura del primer item
    print("\n📋 Estructura del primer restaurante:")
    first_item = dataset[0]
    for key, value in first_item.items():
        print(f"  {key}: {type(value).__name__} = {str(value)[:80]}...")
else:
    print("No hay datos para explorar")

🔍 EXPLORACIÓN DE DATOS OBTENIDOS
No hay datos para explorar


# PROCESAR LOS RESTAURANTES OBTENIDOS Y GUARDARLOS EN LA BD 

In [6]:
# Procesar datos para la BD
if dataset:
    restaurantes_data = []
    
    for item in dataset:
        # Función de limpieza para rango_precio
        def clean_price_level(price_level):
            if price_level is None:
                return ''
            elif isinstance(price_level, (int, float)):
                return str(price_level)
            elif isinstance(price_level, str):
                return price_level
            else:
                return str(price_level)
        
        # Convertir listas a strings
        platos = item.get('dishes', '')
        if isinstance(platos, list):
            platos = ', '.join(platos)
        
        tipo_cocina = extract_cuisines(item.get('cuisines', []))
        if isinstance(tipo_cocina, list):
            tipo_cocina = ', '.join(tipo_cocina)
        
        restaurante = {
            'tripadvisor_id' : item.get('id', ''),
            'nombre' : item.get('name',''),
            'telefono' : item.get('phone',''),
            'direccion' : item.get('address',''),
            'localizacion' : item.get('locationString',''),
            'email' : item.get('email',''),
            'latitud' : item.get('latitude',''),
            'longitud' : item.get('longitude',''),
            'website' : extract_url_path(item.get('website','')),                        
            'rating' : item.get('rating',''),
            'ranking' : item.get('rankingPosition',''),
            'rango_precio': clean_price_level(item.get('priceLevel', '')),
            'platos' : platos,
            'tipo_cocina': tipo_cocina,
            'tripadvisor_web' : item.get('webUrl','')
        }
        restaurantes_data.append(restaurante)
    
    df_clean = pd.DataFrame(restaurantes_data)
    
    # Limpieza adicional por si acaso
    df_clean['rango_precio'] = df_clean['rango_precio'].fillna('').astype(str)
    
    print(f"✅ Datos procesados: {len(df_clean)} restaurantes")

In [7]:
# Guardar la información en la base de datos
if 'df_clean' in locals() and not df_clean.empty:
    conn = None
    try:
        print("💾 Guardando en base de datos...")
        conn = get_db_connection()
        
        # VERIFICACIÓN simple
        cursor = conn.cursor()
        cursor.execute("PRAGMA database_list;")
        db_info = cursor.fetchall()
        print("📋 Base de datos conectada:", db_info[0][2])
        
        # Verificar si la tabla existe
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Restaurantes'")
        if cursor.fetchone():
            print("✅ Tabla Restaurantes encontrada")
            
            # Insertar datos
            rows_inserted = insert_dataframe_to_table(df_clean, 'Restaurantes', conn)
            conn.commit()
            print(f"✅ {rows_inserted} filas insertadas")
        else:
            print("❌ Tabla Restaurantes no existe")
        
        if conn:
            conn.close()
        
    except Exception as e:
        print(f"❌ Error guardando en BD: {e}")
        if conn:
            conn.rollback()
else:
    print("No hay datos limpios para guardar")

No hay datos limpios para guardar
